In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../.."))
%load_ext ElasticNotebook

In [2]:
%%RecordEvent
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

In [3]:
%%RecordEvent
iris_X, iris_y = datasets.load_iris(return_X_y=True)

In [4]:
%%RecordEvent
indices = np.random.permutation(len(iris_X))
iris_X_train = iris_X[indices[:-10]]
iris_y_train = iris_y[indices[:-10]]
iris_X_test = iris_X[indices[-10:]]
iris_y_test = iris_y[indices[-10:]]

In [5]:
%%RecordEvent
indices = 123
knn = KNeighborsClassifier()
knn.fit(iris_X_train, iris_y_train)

knn.predict(iris_X_test)

/Users/libilly/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array([2, 2, 2, 1, 0, 1, 0, 0, 2, 2])

In [6]:
%Checkpoint checkpoints/numpy.pickle


monitor overhead: 0.00899648666381836
storage overhead: 7065
total write: 1.5432441234588623
total read: 0.3107430934906006
total write: 1.5417730808258057
total read: 0.28031015396118164
file size: 799920000
migration speed (bps): 439013972.9777578
---------------------------
variables to migrate:
indices 38
iris_X_train 4749
iris_y_train 1387
iris_X 5069
knn 8700
iris_y 1467
KNeighborsClassifier 1064
np 89
iris_y_test 344
datasets 100
iris_X_test 589
---------------------------
variables to recompute:
[]
---------------------------
cells to recompute:
[]
Checkpoint saved to: checkpoints/numpy.pickle


In [7]:
import dill
import numpy as np
np.random.seed(0)
dill.dump_session(open("test.pkl", "wb"))

ValueError: ctypes objects containing pointers cannot be pickled

In [2]:
print(np.random.rand())


0.4415092899217963


In [1]:
import dill
dill.load_session(open("test.pkl", "rb"))


In [6]:
import mmap
import copy
gen = (i for i in range(100))
gen2 = copy.copy(gen)


TypeError: cannot pickle 'generator' object

In [3]:
import tensorflow as tf
g = tf.random.Generator.from_seed(1234)


ModuleNotFoundError: No module named 'tensorflow'